In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from polarDensity_helper import Coord_Get, get_header_info
from site_distributions import make_simple_site, combine_sites, plot_density

# Step 0: define a site

This is done with the plotting_show_binID notebook.
Some examples from ELIC:

In [ ]:
lipid = "DPPC"
leaf = "upp"
data_root = Path("/home/ems363/Projects/ELIC_DPPC/elicPE17_DPPC/")
fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

In [ ]:
num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
the_data = np.loadtxt(fpath, skiprows=1)
rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

In [ ]:
onebin = make_simple_site(the_data, 25, 30, 1, Ntheta, dr, dth, exrho, frames, [29], f"M4 site, one bin")
onebin_symmetric = make_simple_site(the_data, 25, 30, 1, Ntheta, dr, dth, exrho, frames, np.arange(5,30,6), "M4 site, one bin, symmetric sites")
fourbins = make_simple_site(the_data, 25, 35, 2, Ntheta, dr, dth, exrho, frames, np.arange(28,30,1), "M4 site, four bins")

symmetricA = make_simple_site(the_data, 25, 35, 1, Ntheta, dr, dth, exrho, frames, np.arange(5,30,6), "M4 site, four bins")
symmetricB = make_simple_site(the_data, 25, 35, 1, Ntheta, dr, dth, exrho, frames, np.arange(4,30,6), "M4 site, four bins")
fourbins_symmetric = combine_sites([symmetricA, symmetricB], exrho, "Four bins, symmetric")

extrabin = make_simple_site(the_data, 20, 25, 1, Ntheta, dr, dth, exrho, frames, [28])
fivebins = combine_sites([extrabin, fourbins], exrho, "Five bins")

symmetricextra = make_simple_site(the_data, 20, 25, 1, Ntheta, dr, dth, exrho, frames, np.arange(4,30,6))
fivebins_symmetric = combine_sites([symmetricA, symmetricB, symmetricextra], exrho, "Five bins, symmetric")

sites = {'onebin':onebin,
         'onebin_symmetric': onebin_symmetric, 
         'fourbins':fourbins, 
         'fourbins_symmetric':fourbins_symmetric,
         'fivebins': fivebins,
         'fivebins_symmetric': fivebins_symmetric,
         }

# Step 1 get the total area of the site in question

In [ ]:
for name, site in sites.items():
    print(f"{name:<20}:{np.round(site.area,1):>5}")

# Step 2 get the density 
In VMD:
1. load the trajectory of the empty membrane
2. modify do_get_counts.tcl to have the total area of the site
3. source do_get_counts.tcl
4. It will save to a file called counts_[area].out
5. Provide the path here:

In [ ]:
area = 125
bulk_counts_path = Path(f"/home/ems363/Projects/Polar_Binning_DeltaG/accessible_area/counts_{area}.out")
bulk_counts = np.loadtxt(bulk_counts_path)

frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(range(len(probabilities)), probabilities)
ax.set_ylabel("Probability")
ax.set_xlabel(f"Number of beads in an area about {area} "+r"$\AA^2$")
bulk_mode = np.argmax(probabilities)
ax.vlines([bulk_mode], 0, np.max(probabilities), color = 'black', linestyles='dashed', label = f"mode={bulk_mode}")
ax.legend()

# Compare the bulk mode with the site mode

In [ ]:
rows = 3
cols = 2
fig, axes = plt.subplots(rows,cols, figsize=(15,15))

for site, ax in zip(sites.values(), axes.flatten()):
    ax = plot_density(site, ax)

plt.savefig("Raw_distributions.pdf")

In [ ]:
expt_modes = {}
for key, site in sites.items():
    counts = site.counts.astype(int)
    bincounts = np.bincount(counts.flatten())
    expt_modes[key] = np.argmax(bincounts)

# The accessible area is...
ASSUMING $\left(A_\mathrm{site} \approx A_\mathrm{bulk}\right) \forall A_\mathrm{site}$

$A_\mathrm{acc} = A_\mathrm{total} \frac{M_\mathrm{site}}{M_\mathrm{bulk}}$

In [ ]:
print(f"{'Site Name':<20}:{'Accessible Area':>5}")
for name, site in sites.items():
    A_acc = site.area * expt_modes[name] / bulk_mode
    print(f"{name:<20}:{np.round(A_acc,1):>5}")

# Replot with new areas:

In [ ]:
for name, site in sites.items():
    site.Npeak = exrho* site.area * expt_modes[name] / bulk_mode

In [ ]:
rows = 3
cols = 2
fig, axes = plt.subplots(rows,cols, figsize=(15,15))

for site, ax in zip(sites.values(), axes.flatten()):
    ax = plot_density(site, ax)

plt.savefig(f"replotted_with_corrected_expectation_area{area}.pdf")